In [37]:
import math
import numpy as np
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

## Import necessary items from Keras
import keras
from keras.models import Sequential
from keras.layers import Activation, Dropout, UpSampling2D
from keras.layers import Conv2DTranspose, Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

In [38]:
from google.colab import drive
drive.mount("/content/gdrive")
working_dir = "/content/gdrive/MyDrive/4th Year/7th Semester/EE405 Undergraduate Project I/G-08-2023/Shared Items"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [39]:
def create_model(input_shape, pool_size):
    ## Create the actual neural network here
    model = Sequential()

    ## Normalizes incoming inputs. First layer needs the input shape to work
    model.add(BatchNormalization(input_shape=input_shape))

    ## Below layers were re-named for easier reading of model summary; this not necessary
    ## Conv Layer 1
    model.add(Conv2D(8, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv1'))

    ## Conv Layer 2
    model.add(Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv2'))

    ## Pooling 1
    model.add(MaxPooling2D(pool_size=pool_size))

    ## Conv Layer 3
    model.add(Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv3'))
    model.add(Dropout(0.2))

    ## Conv Layer 4
    model.add(Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv4'))
    model.add(Dropout(0.2))

    ## Conv Layer 5
    model.add(Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv5'))
    model.add(Dropout(0.2))

    ## Pooling 2
    model.add(MaxPooling2D(pool_size=pool_size))

    ## Conv Layer 6
    model.add(Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv6'))
    model.add(Dropout(0.2))

    ## Conv Layer 7
    model.add(Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv7'))
    model.add(Dropout(0.2))

    ## Pooling 3
    model.add(MaxPooling2D(pool_size=pool_size))

    ## Upsample 1
    model.add(UpSampling2D(size=pool_size))

    ## Deconv 1
    model.add(Conv2DTranspose(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv1'))
    model.add(Dropout(0.2))

    ## Deconv 2
    model.add(Conv2DTranspose(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv2'))
    model.add(Dropout(0.2))

    ## Upsample 2
    model.add(UpSampling2D(size=pool_size))

    ## Deconv 3
    model.add(Conv2DTranspose(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv3'))
    model.add(Dropout(0.2))

    ## Deconv 4
    model.add(Conv2DTranspose(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv4'))
    model.add(Dropout(0.2))

    ## Deconv 5
    model.add(Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv5'))
    model.add(Dropout(0.2))

    ## Upsample 3
    model.add(UpSampling2D(size=pool_size))

    ## Deconv 6
    model.add(Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv6'))

    ## Final layer - only including one channel so 1 filter
    model.add(Conv2DTranspose(1, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Final'))

    return model

In [40]:
def main():
    ## Load training images
    # train_images = pickle.load(open(f"{working_dir}/Datasets/Pickle/full_CNN_train.p", "rb" ))
    train_images = pickle.load(open(f"{working_dir}/Datasets/Pickle/test_images_new_train.p", "rb" ))

    ## Load image labels
    # labels = pickle.load(open(f"{working_dir}/Datasets/Pickle/full_CNN_labels.p", "rb" ))
    labels = pickle.load(open(f"{working_dir}/Datasets/Pickle/test_masks_new_train.p", "rb" ))

    ## Make into arrays as the neural network wants these
    train_images = np.array(train_images)
    labels = np.array(labels)

    ## Try to fix TypeError: Cannot convert 0.11764705882352941 to EagerTensor of dtype int64
    # labels = labels.astype('float32')

    ## Normalize labels - training images get normalized to start in the network
    labels = labels / 255

    ## Shuffle images along with their labels, then split into training/validation sets
    train_images, labels = shuffle(train_images, labels)
    ## Test size may be 10% or 20%
    X_train, X_val, y_train, y_val = train_test_split(train_images, labels, test_size=0.1)

    ## Batch size, epochs and pool size below are all paramaters to fiddle with for optimization
    batch_size = 128
    epochs = 150
    pool_size = (2, 2)
    input_shape = X_train.shape[1:]

    ## Create the neural network
    # model = create_model(input_shape, pool_size)
    ## Load existing model
    model = keras.models.load_model(f"{working_dir}/Models/full_CNN_model.h5")
    ## Freeze layers
    for layer in model.layers[:-2]:
        layer.trainable = False
    # model.summary()

    ## Using a generator to help the model use less data
    ## Channel shifts help with shadows slightly
    datagen = ImageDataGenerator(channel_shift_range=0.2)
    ## Augment dataset
    datagen = ImageDataGenerator(
        rotation_range=5,               # Rotate the image 5 degrees in each direction
        # width_shift_range=0.1,          # Skew the image
        # height_shift_range=0.1,         # Skew the image
        horizontal_flip=True,           # Mirror the image
        brightness_range=[0.8, 1.2],    # Slightly increase and decrease exposure
        channel_shift_range=0.2
    )
    datagen.fit(X_train)

    ## Compiling and training the model
    model.compile(optimizer='Adam', loss='mean_squared_error')
    steps_per_epoch = math.ceil(len(X_train) / batch_size)
    model.fit(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=steps_per_epoch,
    epochs=epochs, verbose=1, validation_data=(X_val, y_val))

    ## Freeze layers after training is done
    model.trainable = False
    model.compile(optimizer='Adam', loss='mean_squared_error')

    ## Save model architecture and weights
    model.save('/content/gdrive/MyDrive/4th Year/7th Semester/EE405 Undergraduate Project I/G-08-2023/Shared Items/Models/test_model_new.h5')

    ## Show summary of model
    # model.summary()

if __name__ == '__main__':
    main()

Epoch 1/150
2/2 [==============================] - 3s 433ms/step - loss: 0.1500 - val_loss: 0.1159
Epoch 2/150
2/2 [==============================] - 1s 407ms/step - loss: 0.1338 - val_loss: 0.1064
Epoch 3/150
2/2 [==============================] - 1s 564ms/step - loss: 0.1197 - val_loss: 0.1004
Epoch 4/150
2/2 [==============================] - 1s 402ms/step - loss: 0.1131 - val_loss: 0.0970
Epoch 5/150
2/2 [==============================] - 1s 620ms/step - loss: 0.1056 - val_loss: 0.0958
Epoch 6/150
2/2 [==============================] - 2s 626ms/step - loss: 0.1029 - val_loss: 0.0960
Epoch 7/150
2/2 [==============================] - 2s 640ms/step - loss: 0.1016 - val_loss: 0.0968
Epoch 8/150
2/2 [==============================] - 1s 419ms/step - loss: 0.1026 - val_loss: 0.0975
Epoch 9/150
2/2 [==============================] - 1s 578ms/step - loss: 0.1006 - val_loss: 0.0978
Epoch 10/150
2/2 [==============================] - 1s 407ms/step - loss: 0.1017 - val_loss: 0.0978
Epoch 11/